# MusicGen-Large LoRA ファインチューニング (A100推奨)

**目的**: 大規模データセット（約18,000曲）を使い、MusicGen-Large に LoRA を適用して学習します。
Google Colabのディスク容量制限を回避するため、**順次処理戦略（抽出→トークン化→削除）**を採用しています。

---

## 📋 事前準備

1. **Google Colabのランタイム**: GPU (A100推奨) を選択
2. **シークレットの設定**: Colabの「🔑シークレット」メニューから以下を設定
   - `WANDB_API_KEY`: WandB APIキー
   - `HF_TOKEN`: Hugging Face トークン
3. **Google Drive**: データセット（`archive_batch_xxxx.zip` 形式）を配置

---

## 🚀 使い方

1. セルを順番に実行
2. シークレットから自動的にAPIキーを取得
3. WandBで学習進捗をリアルタイム監視: https://wandb.ai/

# セル 1 — 依存関係インストール

In [ ]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 367.6 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 246.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 403.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 532.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 696.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 542.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 417.5 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 406.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 143.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 283.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 359.5 MB/s  0:00:00
     ━━━━━

--- After initial torch install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Installing xformers...
Looking in indexes: https://download.pytorch.org/whl/cu121
--- After xformers install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/torch/__init__.py", line 1954, in <module>
    _C._initExtension(_manager_path())
  File "/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py", line 264, in <module>
    _lazy_call(_check_capability)
  File "/usr/local/lib/python3.12/dist-packages/tor

In [2]:
# @title download and install

!cd {ROOT_DIR}
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'
!git clone {AUDIOCRAFT_REPO}
!cd {AUDIOCRAFT_DIR}
!pip install -q -e .
!pip install -q dora-search numba wandb

# ^^^ @baltigor


!python -m pip install -U pip setuptools wheel
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

!python -m pip install laion-clap
!python -m pip install -U transformers==4.30.0

fatal: destination path 'audiocraft' already exists and is not an empty directory.
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.11.12-py3-none-any.whl size=3036379 sha256=b50e76e6643050cd795dddfdbb68c00629345a70781f325965d8500f3dce6689
  Stored in directory: /tmp/pip-ephem-wheel-cache-iz05l46r/wheels/b6/70/13/8d2d11b326f983030b72df6408392d5c1b3bc27a9db8b7c5b0
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.11.12
    Uninstalling yt-dlp-2025.11.12:
      Successfully uninstalled yt-dlp-2025.11.12
  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB

# セル 2 — Drive マウントと設定

Google Driveをマウントし、データセットのパスを設定します。

In [38]:
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive',force_remount=True)

# 設定
# ZIPファイルが格納されているディレクトリ
DRIVE_ARCHIVE_DIR = '/content/drive/MyDrive/Archive_Wavs'

# 一時展開先（Colabローカル）
TEMP_DATA_DIR = '/content/temp_dataset'

# トークン保存先（Google Drive推奨、または容量があればローカル）
# ※再開可能にするためDrive推奨ですが、I/Oが遅い場合はローカルにして最後にバックアップする戦略も可
# ここではDriveに直接保存する設定にします
TOKEN_DIR = '/content/drive/MyDrive/MusicGen_Tokens'

os.makedirs(TOKEN_DIR, exist_ok=True)
os.makedirs(TEMP_DATA_DIR, exist_ok=True)

print(f'ZIP格納場所: {DRIVE_ARCHIVE_DIR}')
print(f'一時展開先: {TEMP_DATA_DIR}')
print(f'トークン保存先: {TOKEN_DIR}')

# ZIPファイルの確認
zip_files = sorted(list(Path(DRIVE_ARCHIVE_DIR).glob('archive_batch_*.zip')))
print(f'検出されたZIPファイル数: {len(zip_files)}')
if len(zip_files) > 0:
    print(f'例: {zip_files[0].name}')
else:
    print('⚠️ ZIPファイルが見つかりません。パスを確認してください。')

Mounted at /content/drive
ZIP格納場所: /content/drive/MyDrive/Archive_Wavs
一時展開先: /content/temp_dataset
トークン保存先: /content/drive/MyDrive/MusicGen_Tokens
検出されたZIPファイル数: 139
例: archive_batch_0001.zip


# セル 2.5 — WandBとHugging Faceの認証設定

シークレットからAPIキーを取得してログインします。

In [39]:
from google.colab import userdata
import wandb
import os

# シークレットからAPIキーを取得
try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')

    # WandBログイン
    wandb.login(key=WANDB_API_KEY)
    print("✓ WandBにログインしました")

    # Hugging Face トークンを環境変数に設定
    os.environ['HF_TOKEN'] = HF_TOKEN
    print("✓ Hugging Face トークンを設定しました")

except Exception as e:
    print(f"⚠️ 認証エラー: {e}")
    print("Colabの「シークレット」メニューから以下のキーを設定してください:")
    print("  - WANDB_API_KEY: WandBのAPIキー")
    print("  - HF_TOKEN: Hugging FaceのAPIトークン")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


✓ WandBにログインしました
✓ Hugging Face トークンを設定しました


# セル 2.6 — 🧪 少数サンプルでの動作確認テスト

最初のZIPファイルから少数のファイルを抽出して、パイプライン全体の動作確認を行います。

In [40]:
import time
import json
import os
import shutil
from pathlib import Path
import torch
import torchaudio
from audiocraft.models import CompressionModel, MusicGen
from torch.utils.data import Dataset, DataLoader
import loralib as lora
import torch.nn as nn

# テスト設定
TEST_MODE = True
NUM_TEST_SAMPLES = 5
TEST_EPOCHS = 2

print("=" * 60)
print("🧪 少数サンプルテストモード")
print("=" * 60)

# テスト用のディレクトリ
TEST_DIR = '/content/test_workspace'
TEST_WAV_DIR = os.path.join(TEST_DIR, 'wavs')
TEST_TOKEN_DIR = os.path.join(TEST_DIR, 'tokens')
os.makedirs(TEST_WAV_DIR, exist_ok=True)
os.makedirs(TEST_TOKEN_DIR, exist_ok=True)

# 1. データの抽出（最初のZIPから数ファイルだけ）
print("📁 ステップ1: テストデータの準備")
if len(zip_files) > 0:
    target_zip = zip_files[0]
    print(f"  ZIPファイル: {target_zip.name}")
    # 一時的に展開して数ファイルだけコピー
    !mkdir -p /content/temp_extract
    !tar -xzf "{target_zip}" -C /content/temp_extract

    extracted_files = list(Path('/content/temp_extract').rglob('*.wav'))
    print(f"  抽出されたファイル数: {len(extracted_files)}")

    for i, f in enumerate(extracted_files[:NUM_TEST_SAMPLES]):
        shutil.copy(f, TEST_WAV_DIR)
        print(f"  コピー: {f.name}")

    # 掃除
    !rm -rf /content/temp_extract
else:
    print("⚠️ ZIPファイルがないため、ダミーデータを作成します")
    # ダミーWAV作成（正弦波）
    sr = 32000
    for i in range(NUM_TEST_SAMPLES):
        t = torch.linspace(0, 5, sr * 5)
        wav = torch.sin(2 * 3.14159 * 440 * t).unsqueeze(0)
        torchaudio.save(os.path.join(TEST_WAV_DIR, f'test_{i}.wav'), wav, sr)

test_paths = list(Path(TEST_WAV_DIR).glob('*.wav'))
print(f"✓ {len(test_paths)}個のテストファイルを用意")
print()

# 2. トークナイズのテスト
print("🎵 ステップ2: トークナイズのテスト")
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

for i, p in enumerate(test_paths):
    try:
        wav, sr = torchaudio.load(p)
        if sr != 32000:
            resampler = torchaudio.transforms.Resample(sr, 32000)
            wav = resampler(wav)
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        wav = wav.unsqueeze(0).to('cuda')

        with torch.no_grad():
            encoded_frames = compression_model.encode(wav)
            tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

        outp = os.path.join(TEST_TOKEN_DIR, p.stem + '.pt')
        torch.save({'tokens': tokens.cpu(), 'path': str(p)}, outp)
        print(f"  ✓ トークン化完了: {p.name} -> {tokens.shape}")
    except Exception as e:
        print(f"  ✗ エラー: {e}")

del compression_model
torch.cuda.empty_cache()
print()

# 3. データローダのテスト
print("📦 ステップ3: データローダのテスト")
class TestTokenDataset(Dataset):
    def __init__(self, token_dir, max_length=1500):
        self.files = sorted(list(Path(token_dir).glob('*.pt')))
        self.max_length = max_length
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        tokens = data['tokens']
        if tokens.dim() == 3: tokens = tokens.squeeze(0)
        seq_len = tokens.shape[-1]
        if seq_len > self.max_length:
            tokens = tokens[:, :self.max_length]
        elif seq_len < self.max_length:
            pad = self.max_length - seq_len
            tokens = torch.nn.functional.pad(tokens, (0, pad), value=0)
        return tokens

test_dataset = TestTokenDataset(TEST_TOKEN_DIR)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)
for batch in test_loader:
    print(f"✓ バッチ形状: {batch.shape}")
    break
print()

# 4. モデルとLoRAのテスト
print("🤖 ステップ4: モデルロードとLoRA適用のテスト")
model = MusicGen.get_pretrained('facebook/musicgen-large')
model.to('cuda')
model.eval()

def apply_lora_to_linear(module, r=8, lora_alpha=32, lora_dropout=0.1, parent_name='', count=[0]):
    for name, child in list(module.named_children()):
        if isinstance(child, nn.Linear) and count[0] < 3:  # テスト用制限
            lora_linear = lora.Linear(child.in_features, child.out_features, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout, bias=(child.bias is not None))
            lora_linear.weight.data = child.weight.data.clone()
            if child.bias is not None: lora_linear.bias.data = child.bias.data.clone()
            setattr(module, name, lora_linear)
            count[0] += 1
            print(f"  ✓ LoRA適用: {parent_name}.{name}")
        else:
            apply_lora_to_linear(child, r, lora_alpha, lora_dropout, f"{parent_name}.{name}" if parent_name else name, count)

apply_lora_to_linear(model.lm, r=8)
print("✓ LoRA適用完了")
print()

# 5. 学習ループのテスト
print("🏋️ ステップ5: 学習ループのテスト")
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
model.train()
for epoch in range(TEST_EPOCHS):
    for batch in test_loader:
        batch = batch.to('cuda')
        out = model.lm.forward(batch)
        loss = criterion(out, batch.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"  Epoch {epoch+1}, Loss: {loss.item():.4f}")
        break
print()

# 6. 生成テスト
print("🎶 ステップ6: 生成テスト")
model.eval()
model.set_generation_params(duration=5)
with torch.no_grad():
    wav = model.generate(["Test melody"])
print(f"✓ 生成完了: {wav.shape}")

del model
torch.cuda.empty_cache()
print("✅ 全テスト完了")

ModuleNotFoundError: No module named 'audiocraft.models'

# セル 3.5 — トークナイズ速度計測

最初の100ファイルで処理速度を計測し、全データセットの処理時間を推定します。


In [ ]:
import time
import os
import shutil
from pathlib import Path
import torch
import torchaudio
from tqdm import tqdm
from audiocraft.models import CompressionModel

print("⏱️ トークナイズ速度計測")
print("=" * 60)

# モデルロード
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

# テスト用サンプル数
NUM_SAMPLES = 100

# 最初のZIPファイルからサンプルを抽出
if len(zip_files) > 0:
    target_zip = zip_files[0]
    print(f"📦 テストZIP: {target_zip.name}")

    # 一時展開
    test_extract_dir = '/content/temp_speed_test'
    if os.path.exists(test_extract_dir):
        shutil.rmtree(test_extract_dir)
    os.makedirs(test_extract_dir, exist_ok=True)

    !tar -xzf "{target_zip}" -C "{test_extract_dir}"

    wav_files = list(Path(test_extract_dir).rglob('*.wav'))[:NUM_SAMPLES]
    print(f"📊 テストファイル数: {len(wav_files)}")

    # 速度計測
    start_time = time.time()
    processed = 0

    for wav_path in tqdm(wav_files, desc="計測中"):
        try:
            wav, sr = torchaudio.load(str(wav_path))
            if sr != 32000:
                resampler = torchaudio.transforms.Resample(sr, 32000)
                wav = resampler(wav)
            if wav.shape[0] > 1:
                wav = wav.mean(dim=0, keepdim=True)
            wav = wav.unsqueeze(0).to('cuda')

            with torch.no_grad():
                encoded_frames = compression_model.encode(wav)
                tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

            processed += 1
        except Exception as e:
            print(f"エラー: {wav_path.name} - {e}")
            continue

    elapsed = time.time() - start_time
    files_per_sec = processed / elapsed if elapsed > 0 else 0

    # 全ファイル数の推定
    total_zips = len(zip_files)
    estimated_files_per_zip = len(list(Path(test_extract_dir).rglob('*.wav')))
    total_estimated_files = total_zips * estimated_files_per_zip

    # 推定時間計算
    estimated_total_time = (total_estimated_files / files_per_sec) / 3600  # 時間単位

    print("\n" + "=" * 60)
    print("📈 計測結果")
    print("=" * 60)
    print(f"処理速度: {files_per_sec:.2f} ファイル/秒")
    print(f"推定総ファイル数: {total_estimated_files:,} ファイル")
    print(f"推定総処理時間: {estimated_total_time:.2f} 時間")
    print(f"ZIPファイル数: {total_zips}")
    print("=" * 60)

    # クリーンアップ
    shutil.rmtree(test_extract_dir)
else:
    print("⚠️ ZIPファイルが見つかりません")

del compression_model
torch.cuda.empty_cache()
print("\n✅ 速度計測完了")


# セル 3 — 順次処理によるトークナイズ (Sequential Tokenization)

**戦略**: ZIPファイルを1つずつ展開 → トークン化 → WAV削除 を繰り返します。
これにより、Colabのディスク容量を圧迫せずに大量のデータを処理できます。

In [ ]:
import time
import shutil
import glob
import os
from pathlib import Path
import torch
import torchaudio
from tqdm import tqdm
from audiocraft.models import CompressionModel
import wandb

# WandB初期化
try:
    from google.colab import userdata
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    if not wandb.api.api_key: wandb.login(key=WANDB_API_KEY)
    wandb.init(project='musicgen-lora-finetune', name='sequential_tokenization', job_type='preprocessing')
    use_wandb = True
except: use_wandb = False

# モデルロード
print('EnCodecモデルをロード中...')
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

# 処理済みファイルの確認
existing_tokens = set([Path(f).stem for f in glob.glob(os.path.join(TOKEN_DIR, '*.pt'))])
print(f'既にトークン化済み: {len(existing_tokens)} ファイル')

total_processed = 0
start_time = time.time()

# ZIPファイルごとのループ
for zip_idx, zip_file in enumerate(tqdm(zip_files, desc='ZIPバッチ処理')):
    print(f'\n📦 処理中: {zip_file.name} ({zip_idx+1}/{len(zip_files)})')

    # 1. 展開 (Extract)
    if os.path.exists(TEMP_DATA_DIR):
        shutil.rmtree(TEMP_DATA_DIR)
    os.makedirs(TEMP_DATA_DIR, exist_ok=True)

    print('  ↳ 展開中...')
    !tar -xzf "{zip_file}" -C "{TEMP_DATA_DIR}"

    wav_files = list(Path(TEMP_DATA_DIR).rglob('*.wav'))
    print(f'  ↳ {len(wav_files)} 個のWAVファイルを検出')

    # 2. トークン化 (Tokenize)
    batch_processed = 0
    for wav_path in tqdm(wav_files, desc='  ↳ トークン化', leave=False):
        if wav_path.stem in existing_tokens:
            continue

        try:
            wav, sr = torchaudio.load(str(wav_path))
            if sr != 32000:
                resampler = torchaudio.transforms.Resample(sr, 32000)
                wav = resampler(wav)
            if wav.shape[0] > 1:
                wav = wav.mean(dim=0, keepdim=True)
            wav = wav.unsqueeze(0).to('cuda')

            with torch.no_grad():
                encoded_frames = compression_model.encode(wav)
                tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

            outp = os.path.join(TOKEN_DIR, wav_path.stem + '.pt')
            torch.save({'tokens': tokens.cpu(), 'path': str(wav_path)}, outp)

            batch_processed += 1
            total_processed += 1

        except Exception as e:
            print(f'Error: {wav_path.name} - {e}')
            continue

    # 3. クリーンアップ (Cleanup)
    print('  ↳ クリーンアップ中...')
    shutil.rmtree(TEMP_DATA_DIR)

    # ログ
    if use_wandb:
        wandb.log({
            'processed_files': total_processed,
            'processed_zips': zip_idx + 1,
            'files_in_batch': batch_processed
        })

print(f'\n✅ 全処理完了: {total_processed} ファイルを新規にトークン化しました')
if use_wandb: wandb.finish()

del compression_model
torch.cuda.empty_cache()

# セル 4 — DeepSpeed 設定ファイル作成

In [ ]:
import json

ds_config = {
  "train_batch_size": 16,
  "train_micro_batch_size_per_gpu": 2,
  "gradient_accumulation_steps": 8,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 1e-4,
      "weight_decay": 0.01
    }
  },
  "fp16": {
    "enabled": True
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": True
    },
    "offload_param": {
      "device": "cpu"
    },
    "overlap_comm": True,
    "contiguous_gradients": True
  }
}

with open('ds_config.json','w') as f:
    json.dump(ds_config, f, indent=2)

print('ds_config.json を作成しました')

# セル 5 — モデル準備とLoRA適用

In [ ]:
import loralib as lora
import torch.nn as nn
import torch
from audiocraft.models import MusicGen

# モデルロード
print('MusicGen-Large をロード中...')
model = MusicGen.get_pretrained('facebook/musicgen-large')
model.eval()

# LoRA適用関数
def apply_lora_to_linear(module, r=8, lora_alpha=32, lora_dropout=0.1, parent_name=''):
    for name, child in list(module.named_children()):
        full_name = f"{parent_name}.{name}" if parent_name else name
        if isinstance(child, nn.Linear):
            lora_linear = lora.Linear(
                child.in_features, child.out_features, r=r,
                lora_alpha=lora_alpha, lora_dropout=lora_dropout, bias=(child.bias is not None)
            )
            lora_linear.weight.data = child.weight.data.clone()
            if child.bias is not None:
                lora_linear.bias.data = child.bias.data.clone()
            setattr(module, name, lora_linear)
            print(f'Applied LoRA to: {full_name}')
        else:
            apply_lora_to_linear(child, r, lora_alpha, lora_dropout, full_name)

print('\nLoRAを適用中...')
apply_lora_to_linear(model.lm, r=8)

# 勾配設定
for p in model.parameters(): p.requires_grad = False
for name, p in model.named_parameters():
    if 'lora_' in name: p.requires_grad = True

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'\n学習可能パラメータ: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)')

# セル 6 — データローダ作成

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from pathlib import Path
import random

class TokenDataset(Dataset):
    def __init__(self, token_dir, max_length=1500):
        self.files = sorted(list(Path(token_dir).glob('*.pt')))
        self.max_length = max_length
        print(f'Dataset initialized with {len(self.files)} samples')

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        tokens = data['tokens']
        if tokens.dim() == 3: tokens = tokens.squeeze(0)

        seq_len = tokens.shape[-1]
        if seq_len > self.max_length:
            start = random.randint(0, seq_len - self.max_length)
            tokens = tokens[:, start:start + self.max_length]
        elif seq_len < self.max_length:
            pad = self.max_length - seq_len
            tokens = torch.nn.functional.pad(tokens, (0, pad), value=0)
        return tokens

dataset = TokenDataset(TOKEN_DIR, max_length=1500)
train_size = int(0.95 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, num_workers=2, pin_memory=True)

print(f'Train: {len(train_dataset)}, Valid: {len(valid_dataset)}')

In [ ]:
# 最終モデル保存（学習完了後）
print('💾 最終モデルを保存中...')
final_checkpoint = {
    'epoch': num_epochs - 1,
    'global_step': global_step,
    'model_state': model.state_dict(),
    'opt_state': optimizer.state_dict(),
    'loss': avg_loss,
    'timestamp': time.strftime("%Y%m%d_%H%M%S")
}
final_lora_checkpoint = {
    'epoch': num_epochs - 1,
    'global_step': global_step,
    'lora_weights': extract_lora_weights(model),
    'loss': avg_loss,
    'timestamp': final_checkpoint['timestamp']
}

torch.save(final_checkpoint, os.path.join(FULL_MODEL_DIR, 'final_model.pt'))
torch.save(final_lora_checkpoint, os.path.join(LORA_DIR, 'final_lora.pt'))
print('✓ 最終モデルを保存しました')

# WandBアーティファクトとして保存
if use_wandb:
    try:
        if os.path.exists(os.path.join(LORA_DIR, 'best_lora.pt')):
            artifact = wandb.Artifact('best_lora_weights', type='model')
            artifact.add_file(os.path.join(LORA_DIR, 'best_lora.pt'))
            wandb.log_artifact(artifact)
            print('✓ ベストLoRA重みをWandBに保存しました')

        artifact_final = wandb.Artifact('final_lora_weights', type='model')
        artifact_final.add_file(os.path.join(LORA_DIR, 'final_lora.pt'))
        wandb.log_artifact(artifact_final)
        print('✓ 最終LoRA重みをWandBに保存しました')

        wandb.finish()
        print('✓ WandBセッションを終了しました')
    except Exception as e:
        print(f'⚠️ WandBアーティファクト保存エラー: {e}')

print('✅ 全処理完了')


# セル 7 — 学習ループ (WandB + Checkpoints)

In [ ]:
import os
import time
import torch
import torch.nn as nn
import wandb
from tqdm import tqdm
from google.colab import userdata

# WandB初期化
try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    if not wandb.api.api_key: wandb.login(key=WANDB_API_KEY)
    wandb.init(project='musicgen-lora-finetune', name='training_run', tags=['musicgen', 'lora', 'a100'])
    use_wandb = True
except: use_wandb = False

device = torch.device('cuda')
model = model.to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=0.01)

# 保存先設定
CKPT_DIR = '/content/drive/MyDrive/MusicGen_Checkpoints'
LORA_DIR = os.path.join(CKPT_DIR, 'lora_weights')
FULL_MODEL_DIR = os.path.join(CKPT_DIR, 'full_models')
os.makedirs(LORA_DIR, exist_ok=True)
os.makedirs(FULL_MODEL_DIR, exist_ok=True)

def extract_lora_weights(model):
    return {name: param.cpu().detach().clone() for name, param in model.named_parameters() if 'lora_' in name and param.requires_grad}

def save_checkpoint(model, optimizer, epoch, global_step, loss, checkpoint_type='regular'):
    checkpoint = {
        'epoch': epoch, 'global_step': global_step, 'model_state': model.state_dict(),
        'opt_state': optimizer.state_dict(), 'loss': loss, 'best_loss': best_loss,
        'timestamp': time.strftime("%Y%m%d_%H%M%S")
    }
    lora_checkpoint = {
        'epoch': epoch, 'global_step': global_step, 'lora_weights': extract_lora_weights(model),
        'loss': loss, 'timestamp': checkpoint['timestamp']
    }

    if checkpoint_type == 'regular':
        torch.save(checkpoint, os.path.join(CKPT_DIR, 'latest.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, 'latest_lora.pt'))
    elif checkpoint_type == 'best':
        torch.save(checkpoint, os.path.join(FULL_MODEL_DIR, 'best_model.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, 'best_lora.pt'))
        print(f'  🏆 ベストモデル保存: Loss {loss:.4f}')
    elif checkpoint_type == 'epoch':
        torch.save(checkpoint, os.path.join(FULL_MODEL_DIR, f'checkpoint_epoch_{epoch+1}.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, f'lora_epoch_{epoch+1}.pt'))
    elif checkpoint_type == 'step':
        torch.save(checkpoint, os.path.join(CKPT_DIR, f'checkpoint_step_{global_step}.pt'))

# 再開処理
start_epoch = 0
global_step = 0
best_loss = float('inf')
latest_ckpt = os.path.join(CKPT_DIR, 'latest.pt')
if os.path.exists(latest_ckpt):
    print(f'🔄 再開: {latest_ckpt}')
    ckpt = torch.load(latest_ckpt, map_location='cpu')
    model.load_state_dict(ckpt['model_state'], strict=False)
    optimizer.load_state_dict(ckpt['opt_state'])
    start_epoch = ckpt['epoch'] + 1
    global_step = ckpt.get('global_step', 0)
    best_loss = ckpt.get('best_loss', float('inf'))

criterion = nn.MSELoss()
num_epochs = 5
accum_steps = 8

print('\n🚀 学習開始')
for epoch in range(start_epoch, num_epochs):
    model.train()
    epoch_loss = 0.0
    optimizer.zero_grad()

    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch_idx, tokens in enumerate(pbar):
        try:
            tokens = tokens.to(device)
            outputs = model.lm.forward(tokens)
            loss = criterion(outputs, tokens.float()) / accum_steps
            loss.backward()

            if (batch_idx + 1) % accum_steps == 0:
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 1.0)
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

                if use_wandb and global_step % 10 == 0:
                    wandb.log({'train/loss': loss.item() * accum_steps, 'train/step': global_step})
                if global_step % 500 == 0:
                    save_checkpoint(model, optimizer, epoch, global_step, loss.item() * accum_steps, 'step')

            epoch_loss += loss.item() * accum_steps
            pbar.set_postfix({'loss': f'{loss.item() * accum_steps:.4f}'})
        except Exception as e:
            print(f'Error: {e}')
            continue

    avg_loss = epoch_loss / len(train_loader)
    print(f'\nEpoch {epoch+1} Loss: {avg_loss:.4f}')

    if use_wandb: wandb.log({'epoch/loss': avg_loss, 'epoch': epoch+1})

    if avg_loss < best_loss:
        best_loss = avg_loss
        save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'best')

    save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'regular')
    save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'epoch')

print('🎉 学習完了')

# セル 8 — 楽曲生成

In [ ]:
from audiocraft.data.audio import audio_write
from IPython.display import Audio, display

print('🎵 楽曲生成')
model.eval()
model.set_generation_params(use_sampling=True, top_k=250, duration=30)

prompts = [
    "A dynamic heavy metal song with fast drums and guitar solo",
    "Relaxing jazz piano with soft background ambience",
    "Upbeat electronic dance music with strong bass"
]

with torch.no_grad():
    wav = model.generate(prompts)

output_dir = '/content/drive/MyDrive/MusicGen_Generated'
os.makedirs(output_dir, exist_ok=True)

for idx, one_wav in enumerate(wav):
    filename = os.path.join(output_dir, f"generated_{idx}")
    audio_write(filename, one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    print(f'\nPrompt: {prompts[idx]}')
    display(Audio(filename + ".wav"))

# Task
ランタイムが途中で停止したため、中断した処理を再開できません。まずは、手動でランタイムを再起動してください。

**操作手順**:
1.  Colabのメニューから「ランタイム」を選択します。
2.  「ランタイムを再起動」をクリックします。

ランタイムの再起動が完了したら、以下のセルを順番に実行して環境設定を再度行ってください。

*   **`hoPXeER8ZYvm`**: 必要なPythonパッケージ（`torch`, `torchaudio`, `torchvision`, `xformers`など）をインストールします。
*   **`bemXRN_8Zgrh`**: Audiocraftおよびその他の必要なライブラリ（`dora-search`, `numba`, `wandb`, `yt-dlp`, `laion-clap`, `transformers`）をインストールします。
*   **`53bd8d67`**: Google Driveをマウントし、データセットのパス (`DRIVE_ARCHIVE_DIR`, `TEMP_DATA_DIR`, `TOKEN_DIR`) が設定されていることを確認します。
*   **`0df137c8`**: WandBとHugging FaceのAPIキーが正しく設定され、ログインが完了していることを確認します。
*   **`a3cdaf43`**: ファインチューニングパイプラインの動作を少数サンプルで確認します。

これらのステップが完了し、テストが成功したことを確認してください。

## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。


## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。

#### Instructions
1. Colabのメニューバーから「ランタイム」をクリックします。
2. ドロップダウンメニューから「ランタイムを再起動」を選択し、クリックします。
3. 確認メッセージが表示されたら、「はい」または「Restart runtime」をクリックしてランタイムの再起動を実行します。
（注意: この操作はAIエージェントからは実行できません。手動でお願いします。）

## 依存関係の再インストール

### Subtask:
ランタイムが再起動されたら、再度セルhoPXeER8ZYvmを実行して、必要なPythonパッケージ（torch, torchaudio, torchvision, xformersなど）をインストールします。これにより、環境の整合性を確保します。


**Reasoning**:
The subtask explicitly states to re-execute cell `hoPXeER8ZYvm` to install necessary Python packages after a potential runtime restart. This cell handles the installation of torch, torchaudio, torchvision, xformers, timm, and fastai.



In [ ]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torch.__version__}")

**Reasoning**:
The subtask explicitly states to re-execute cell `hoPXeER8ZYvm` to install necessary Python packages after a potential runtime restart. This cell handles the installation of torch, torchaudio, torchvision, xformers, timm, and fastai.



In [1]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 319.0 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 373.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 283.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 288.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 663.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 350.0 MB/s  0:00:00
     

--- After initial torch install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Installing xformers...
Looking in indexes: https://download.pytorch.org/whl/cu121
--- After xformers install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
xFormers 0.0.29.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@v2.5.7-pt:         available
memory_efficient_attention.fa2B@v2.5.7-pt:         available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.triton_splitKF:         available
indexing

**Reasoning**:
The previous step successfully reinstalled the core Python packages. Now, to continue the environment setup as per the instructions, the next logical step is to install Audiocraft and other related libraries by executing cell `bemXRN_8Zgrh`.



In [3]:
ROOT_DIR = f'/content'

!cd {ROOT_DIR}
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'
!git clone {AUDIOCRAFT_REPO}
!cd {AUDIOCRAFT_DIR}
!pip install -q -e .
!pip install -q dora-search numba wandb

# ^^^ @baltigor


!python -m pip install -U pip setuptools wheel
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

!python -m pip install laion-clap
!python -m pip install -U transformers==4.30.0

fatal: destination path 'audiocraft' already exists and is not an empty directory.
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
laion-clap 1.1.7 requires numpy<2.0.0,>=1.23.5, but you have numpy 2.0.2 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 70.2.0
    Uninstalling setuptools-70.2.0:
      Successfully uninstalled setuptools-70.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the 

**Reasoning**:
The previous installation of `audiocraft` failed because the directory already existed and the `pip install -e .` command was not executed in the correct directory. I also need to ensure that `audiocraft` is installed before proceeding with other package installations that might depend on it, especially since the `ModuleNotFoundError: No module named 'audiocraft.models'` occurred in a subsequent cell. I will also try to force install `tokenizers` from its wheel to avoid build issues. The current installation attempts show multiple dependency conflicts; I need to ensure a clean and correct installation sequence.



In [5]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install audiocraft in editable mode, ensuring to execute pip from within the directory
!pip install -q -e "{AUDIOCRAFT_DIR}"

# Install other dependencies
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
!python -m pip install laion-clap

# Install specific version of transformers, attempting to force wheel if build failed previously
!pip install -U --force-reinstall tokenizers==0.13.3
!pip install -U --force-reinstall transformers==4.30.0

print("All specified dependencies installed.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (1198/1198), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
ERROR: Failed to build 'av' when getting requirements to build wheel
  Using cached https://github.com/yt-dlp/yt

# Task
Okay, I will proceed to execute the next cell in the plan: `bemXRN_8Zgrh`. This cell will install Audiocraft and other necessary libraries.

## Audiocraftのインストールとその他の依存関係

### Subtask:
セル `bemXRN_8Zgrh` を実行し、Audiocraftおよびその他の必要なライブラリ（`dora-search`, `numba`, `wandb`, `yt-dlp`, `laion-clap`, `transformers`）をインストールします。

## 依存関係の再インストール

### Subtask:
ランタイムが再起動されたら、再度セルhoPXeER8ZYvmを実行して、必要なPythonパッケージ（torch, torchaudio, torchvision, xformersなど）をインストールします。これにより、環境の整合性を確保します。


**Reasoning**:
The subtask explicitly states to re-execute cell `hoPXeER8ZYvm` to install necessary Python packages after a potential runtime restart. This cell handles the installation of torch, torchaudio, torchvision, xformers, timm, and fastai.



In [1]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 448.8 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 176.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 335.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 249.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 203.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 262.3 MB/s  0:00:00
     

--- After initial torch install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Installing xformers...
Looking in indexes: https://download.pytorch.org/whl/cu121


KeyboardInterrupt: 

**Reasoning**:
The previous execution of this cell was interrupted by a `KeyboardInterrupt` during the installation of `xformers`. Re-executing the cell will ensure all required Python packages are installed correctly to proceed with the notebook.



In [1]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("---" After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torch.__version__}")

SyntaxError: unterminated string literal (detected at line 30) (ipython-input-2830021825.py, line 30)

**Reasoning**:
The previous code failed due to a `SyntaxError: unterminated string literal` on line 30. I will fix the print statement to correctly enclose the string in quotes, then re-execute the cell to ensure all packages are installed correctly.



In [ ]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 488.5 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 138.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 529.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 451.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 676.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 385.5 MB/s  0:00:00
     

**Reasoning**:
The previous execution of this cell was interrupted by a `KeyboardInterrupt` during the installation of `xformers`, and the print statement also had a `SyntaxError`. I have corrected the `SyntaxError` by enclosing the string properly in quotes and will re-execute the cell to ensure all required Python packages are installed correctly to proceed with the notebook.



In [2]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 373.7 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 388.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 249.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 273.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 623.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 359.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 293.5 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 315.9 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 352.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 335.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 240.8 MB/s  0:00:00
     ━━━━━

--- After initial torch install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Installing xformers...
Looking in indexes: https://download.pytorch.org/whl/cu121
--- After xformers install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
xFormers 0.0.29.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@v2.5.7-pt:         available
memory_efficient_attention.fa2B@v2.5.7-pt:         available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.triton_splitKF:         available
indexing

**Reasoning**:
The previous step successfully reinstalled the core Python packages. Now, to continue the environment setup as per the instructions, the next logical step is to install Audiocraft and other related libraries by executing cell `bemXRN_8Zgrh`.



In [ ]:
ROOT_DIR = f'/content'

!cd {ROOT_DIR}
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'
!git clone {AUDIOCRAFT_REPO}
!cd {AUDIOCRAFT_DIR}
!pip install -q -e .
!pip install -q dora-search numba wandb

# ^^^ @baltigor


!python -m pip install -U pip setuptools wheel
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

!python -m pip install laion-clap
!python -m pip install -U transformers==4.30.0

**Reasoning**:
The previous attempts to install Audiocraft and its dependencies failed due to directory not found issues and a build error for `tokenizers`. I will ensure a clean Audiocraft clone, install necessary system packages (`ffmpeg`, `libav-tools`), correctly install Audiocraft by chaining commands in a subshell, and then install remaining Python dependencies, letting `transformers` handle its `tokenizers` dependency.



In [1]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing
print("Installing system dependencies (ffmpeg, libav-tools)...")
!apt-get update -qq
!apt-get install -y ffmpeg libav-tools

# Install audiocraft in editable mode, ensuring to execute pip from within the directory
print(f"Installing audiocraft from {AUDIOCRAFT_DIR}...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e .

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 12.53 MiB/s, done.
Resolving deltas: 100% (1198/1198), done.
Installing system dependencies (ffmpeg, libav-tools)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'libav-tools' has no installation candidate
Installing audiocraft from /content/audiocraft...
  Installing build de

**Reasoning**:
The previous attempt to install `audiocraft` failed due to a build error related to `av` and a missing `libav-tools` package (which is deprecated). I will refine the installation by first installing `build-essential` and `pkg-config` as these are often required for `av` to build correctly, then attempt the `audiocraft` installation again, and finally install the remaining Python dependencies. I will also explicitly remove `libav-tools` from the apt-get install line since it's deprecated and causing an error.



In [2]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Install audiocraft in editable mode, ensuring to execute pip from within the directory
print(f"Installing audiocraft from {AUDIOCRAFT_DIR}...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e .

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remo

**Reasoning**:
The previous attempt to install audiocraft failed because its dependency resolver tried to install an incompatible version of torch (2.1.0) while torch 2.5.1 was already installed. To resolve this, I will install audiocraft by explicitly telling pip to ignore its declared dependencies and rely on the already installed torch version. I will also make sure `tokenizers` and `transformers` are installed cleanly after this.



In [3]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Install audiocraft in editable mode, ignoring its dependencies since torch 2.5.1 is already installed.
# This assumes audiocraft is compatible with torch 2.5.1 despite its specified 2.1.0 requirement.
print(f"Installing audiocraft from {AUDIOCRAFT_DIR} (ignoring dependencies)...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e . --no-deps

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 13.63 MiB/s, done.
Resolving deltas: 100% (1198/1198), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavdevice-dev 

# Task
I will re-execute cell `fd15a6ce` to complete the installation of Audiocraft and its dependencies, including `dora-search`, `numba`, `wandb`, `yt-dlp`, `laion-clap`, and `transformers`. This is necessary because the previous execution was interrupted, and some packages like `transformers` (which depends on `tokenizers`) might not have been installed correctly.

```python
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Install audiocraft in editable mode, ignoring its dependencies since torch 2.5.1 is already installed.
# This assumes audiocraft is compatible with torch 2.5.1 despite its specified 2.1.0 requirement.
print(f"Installing audiocraft from {AUDIOCRAFT_DIR} (ignoring dependencies)...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e . --no-deps

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")
```

## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。


## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。

#### Instructions
1. Colabのメニューバーから「ランタイム」をクリックします。
2. ドロップダウンメニューから「ランタイムを再起動」を選択し、クリックします。
3. 確認メッセージが表示されたら、「はい」または「Restart runtime」をクリックしてランタイムの再起動を実行します。
（注意: この操作はAIエージェントからは実行できません。手動でお願いします。）

## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。

#### Instructions
1. Colabのメニューバーから「ランタイム」をクリックします。
2. ドロップダウンメニューから「ランタイムを再起動」を選択し、クリックします。
3. 確認メッセージが表示されたら、「はい」または「Restart runtime」をクリックしてランタイムの再起動を実行します。
（注意: この操作はAIエージェントからは実行できません。手動でお願いします。）

## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。

#### Instructions
1. Colabのメニューバーから「ランタイム」をクリックします。
2. ドロップダウンメニューから「ランタイムを再起動」を選択し、クリックします。
3. 確認メッセージが表示されたら、「はい」または「Restart runtime」をクリックしてランタイムの再起動を実行します。
（注意: この操作はAIエージェントからは実行できません。手動でお願いします。）

## 手動でランタイムを再起動

### Subtask:
Colabのメニューから「ランタイム」→「ランタイムを再起動」を選択し、手動でランタイムを再起動します。

#### Instructions
1. Colabのメニューバーから「ランタイム」をクリックします。
2. ドロップダウンメニューから「ランタイムを再起動」を選択し、クリックします。
3. 確認メッセージが表示されたら、「はい」または「Restart runtime」をクリックしてランタイムの再起動を実行します。
（注意: この操作はAIエージェントからは実行できません。手動でお願いします。）

**Reasoning**:
The user has performed the manual restart. Now, as per the initial instructions in cell `6c3e0ae5` and the original plan, the next step is to re-execute cell `hoPXeER8ZYvm` to reinstall the necessary Python packages (`torch`, `torchaudio`, `torchvision`, `xformers`, `timm`, `fastai`) to ensure a consistent environment after the runtime restart. This cell has been problematic before, so I'm calling it again to ensure proper installation.



In [4]:
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y

# Force-reinstall to ensure the correct version is picked, and clear cache
print("Installing torch, torchaudio, torchvision...")
!pip install --no-cache-dir --force-reinstall torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

# Check immediately after
import torch
import torchaudio
import torchvision

print("--- After initial torch install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

# Install xformers, explicitly telling pip not to install its dependencies (i.e., torch)
# as torch is already installed
print("Installing xformers...")
!pip install --no-deps xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121

# Check again
import torch
import torchaudio
import torchvision

print("--- After xformers install ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")
!python -m xformers.info

# Install other dependencies
print("Installing timm and fastai...")
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5

# Final check
import torch
import torchaudio
import torchvision

print("--- After all installs ---")
print(f"torch: {torch.__version__}")
print(f"torchaudio: {torchaudio.__version__}")
print(f"torchvision: {torchvision.__version__}")

/content
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Installing torch, torchaudio, torchvision...
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 383.0 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 348.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 486.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 387.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 630.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 403.8 MB/s  0:00:00
     

--- After initial torch install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
Installing xformers...
Looking in indexes: https://download.pytorch.org/whl/cu121
--- After xformers install ---
torch: 2.5.1+cu121
torchaudio: 2.5.1+cu121
torchvision: 0.20.1+cu121
xFormers 0.0.29.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@v2.5.7-pt:         available
memory_efficient_attention.fa2B@v2.5.7-pt:         available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.triton_splitKF:         available
indexing

**Reasoning**:
The previous step successfully reinstalled the core Python packages. Now, as per the instructions and to address previous installation issues, I will execute cell `bemXRN_8Zgrh` with modifications to ensure a clean installation of Audiocraft and its remaining dependencies, specifically ignoring Audiocraft's torch dependency to prevent conflicts, and installing relevant system packages first.



In [5]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Install audiocraft in editable mode, ignoring its dependencies since torch 2.5.1 is already installed.
# This assumes audiocraft is compatible with torch 2.5.1 despite its specified 2.1.0 requirement.
print(f"Installing audiocraft from {AUDIOCRAFT_DIR} (ignoring dependencies)...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e . --no-deps

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 14.11 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavdevice-dev 

**Reasoning**:
The previous attempts to install Audiocraft and its dependencies have failed due to unmet dependencies and build errors for `av` and `tokenizers`. To address this, I will implement a more robust installation strategy: first ensure core `av` system libraries are present, then explicitly install the `av` Python package required by `audiocraft`, install `audiocraft` allowing it to handle its remaining dependencies, and finally install other specified Python libraries including `transformers` which will manage `tokenizers`.



In [6]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Explicitly install av==11.0.0 which audiocraft 1.4.0a2 requires
print("Installing av==11.0.0...")
!pip install -q av==11.0.0

# Install audiocraft in editable mode, allowing it to install its remaining dependencies.
# The --no-deps was problematic before, let pip resolve it now after av is installed.
print(f"Installing audiocraft from {AUDIOCRAFT_DIR}...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e .

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp to ensure it's up to date and correct
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency automatically
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavdevice-dev i

**Reasoning**:
The previous installation attempts failed due to dependency conflicts, particularly `audiocraft` requiring `torch==2.1.0` which conflicts with the installed `torch==2.5.1`. I will explicitly install `av==11.0.0` (a direct dependency for audiocraft) and then install `audiocraft` from source using `--no-deps` to avoid re-installing conflicting torch versions. Finally, I will install the remaining Python dependencies ensuring `transformers` handles its `tokenizers` dependency.



In [1]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory if it exists to ensure a clean clone
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Explicitly install av==11.0.0 which audiocraft 1.4.0a2 requires
print("Installing av==11.0.0...")
!pip install -q av==11.0.0

# Install audiocraft in editable mode, allowing it to install its remaining dependencies.
# The --no-deps was problematic before, let pip resolve it now after av is installed.
print(f"Installing audiocraft from {AUDIOCRAFT_DIR}...")
!cd {AUDIOCRAFT_DIR} && pip install -q -e . --no-deps

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp to ensure it's up to date and correct
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency automatically
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 12.17 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavdevice-dev 

**Reasoning**:
The previous attempts to install `audiocraft` failed due to dependency conflicts, specifically its requirement for `torch==2.1.0` while `torch==2.5.1` is already installed. To resolve this, I will try installing `audiocraft` from PyPI (which may provide a more recent, compatible version) instead of the local cloned repository. I will also explicitly install `av==11.0.0` beforehand to satisfy one of `audiocraft`'s reported dependencies, and then proceed with the other installations.



In [3]:
ROOT_DIR = f'/content'
AUDIOCRAFT_DIR = f'{ROOT_DIR}/audiocraft'
AUDIOCRAFT_REPO = f'https://github.com/facebookresearch/audiocraft.git'

# Remove existing audiocraft directory to ensure clean state
!rm -rf {AUDIOCRAFT_DIR}

# Clone audiocraft (but will install from pypi later)
!git clone {AUDIOCRAFT_REPO}

# Install system dependencies for audio processing, including build tools for 'av'
print("Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...")
!apt-get update -qq
!apt-get install -y ffmpeg build-essential pkg-config libavformat-dev libavcodec-dev libavdevice-dev libavfilter-dev libavutil-dev libswresample-dev libswscale-dev

# Explicitly install av==11.0.0 which audiocraft 1.4.0a2 reportedly requires
print("Installing av==11.0.0...")
!pip install -q av==11.0.0

# Install audiocraft from PyPI (this might get a newer, more compatible version)
print("Installing audiocraft from PyPI...")
!pip install -q audiocraft

# Install other dependencies
print("Installing dora-search, numba, wandb...")
!pip install -q dora-search numba wandb

# Update pip, setuptools, wheel
print("Updating pip, setuptools, wheel...")
!python -m pip install -U pip setuptools wheel

# Reinstall yt-dlp to ensure it's up to date and correct
print("Installing yt-dlp...")
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

# Install laion-clap
print("Installing laion-clap...")
!python -m pip install laion-clap

# Install specific version of transformers, this should handle tokenizers as a dependency automatically
print("Installing transformers...")
!python -m pip install -U transformers==4.30.0

print("All specified dependencies installation process initiated.")

Cloning into 'audiocraft'...
remote: Enumerating objects: 2011, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2011 (delta 10), reused 8 (delta 8), pack-reused 1997 (from 2)
Receiving objects: 100% (2011/2011), 24.72 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (1198/1198), done.
Installing system dependencies (ffmpeg, build-essential, pkg-config, libavformat-dev)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavcodec-dev is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
libavdevice-dev 